In [60]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Embedding
from keras.initializers import he_uniform
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from keras import backend as K

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

Избавление от повторов

In [3]:
#df_train['Word'] = df_train.Word.apply(str.lower)
df_train = df_train.groupby('Word', group_keys=True).apply(lambda x: x.loc[x.Label.idxmax()])

Разбиение на train и val

In [4]:
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_val = df_train.iloc[:int(len(df_train)*0.15)]
df_train = df_train.iloc[int(len(df_train)*0.15):]

Tf_idf

In [5]:
df = pd.concat((df_train, df_test))
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1, 4), lowercase=False)
vectorizer.fit(df['Word'])

TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 4), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

Вычисления для downsample-инга

In [6]:
df_train_label_1 = df_train[df_train.Label == 1]
n = df_train_label_1.shape[0]
m = vectorizer.transform(df_train_label_1.Word).shape[1]

C:\Users\Alexey\Miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Векторизация val

In [7]:
X_val = vectorizer.transform(df_val['Word'])
y_val = df_val['Label']

C:\Users\Alexey\Miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


**MLP**

Архитектура MLP (простая т.к. фичи разреженные и предположительно разбить их на 2 класса лучше более линейной гиперплосокстью)

Далее на каждом j идет downsampling к классу 1 и дообучение MLP (ВАЖНО!!! требуется достаточное кол-во свободной оперативном памяти ~ 10ГБ)

При падении AUR ROC SCORE остановка обучения

In [50]:
model_MLP = Sequential()
model_MLP.add(Dense(10, input_dim=m, activation='relu', kernel_initializer='he_uniform'))
model_MLP.add(Dense(1, activation='sigmoid'))
model_MLP.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
for i in range(1):
    prev_score = 0
    print('global epoch =',i)
    for j in range(9):
        print('local epoch =',j)
        if j < 8:
            df_train_downsampled = pd.concat([df_train_label_1, df_train[df_train.Label == 0][j*n:j*n+n]], axis=0)
            df_train_downsampled = df_train_downsampled.sample(frac=1).reset_index(drop=True)
            X_train = vectorizer.transform(df_train_downsampled['Word'])
        else:
            df_train_downsampled = pd.concat([df_train_label_1, df_train[df_train.Label == 0][j*n:]], axis=0)
            df_train_downsampled = df_train_downsampled.sample(frac=1).reset_index(drop=True)
            X_train = vectorizer.transform(df_train_downsampled['Word'])
        
        X_train_mtx = X_train.todense()
        X_val_mtx = X_val.todense()
        
        model_MLP.fit(X_train_mtx, df_train_downsampled.Label, batch_size=10, epochs=1)
        del(X_train_mtx)
        score = roc_auc_score(y_val, model_MLP.predict(X_val_mtx))
        print("val score", roc_auc_score(y_val, model_MLP.predict(X_val_mtx)))
        if score > prev_score:
            prev_score = score
        else:
            print("SCORE WAS DECEREASED!!!!")
            del(X_val_mtx)
            break
        del(X_val_mtx)

global epoch = 0
local epoch = 0


C:\Users\Alexey\Miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Epoch 1/1
18140/18140 [==============================] - 118s 7ms/step - loss: 0.4963 - acc: 0.7915
val score 0.8915326488767519
local epoch = 1
Epoch 1/1
18140/18140 [==============================] - 116s 6ms/step - loss: 0.3454 - acc: 0.8605
val score 0.904226138327357
local epoch = 2
Epoch 1/1
18140/18140 [==============================] - 112s 6ms/step - loss: 0.2752 - acc: 0.8895
val score 0.9079330250646143
local epoch = 3
Epoch 1/1
18140/18140 [==============================] - 115s 6ms/step - loss: 0.2222 - acc: 0.9121
val score 0.9062154121290898
SCORE WAS DECEREASED!!!!


**LSTM**

In [77]:
df_train_downsampled = pd.concat([df_train_label_1, df_train[df_train.Label == 0][:n]], axis=0)
df_train_downsampled = df_train_downsampled.sample(frac=1).reset_index(drop=True)

Создание словаря

In [78]:
str1 = ''.join(str(e) for e in df.Word.tolist())
spl = list(str1)
dicti = np.unique(spl)

One Hot Encoding (OHE)

In [80]:
le = LabelEncoder()
le.fit(dicti)

LabelEncoder()

OHE для train

In [127]:
df_train_LSTM = df_train.copy()
df_train_LSTM.Word = df_train.Word.apply(list)
df_train_LSTM.Word = df_train_LSTM.Word.apply(le.transform)
df_train_LSTM.reset_index(drop=True, inplace=True)

Перевод в матрицу с длиной фич до 20

In [128]:
maxlen = 20
X_tr_lstm = np.zeros((df_train_LSTM.shape[0], maxlen), dtype=int)
for k in range(df_train_LSTM.shape[0]):
    X_tr_lstm[k][:df_train_LSTM.Word[k].shape[0]] = df_train_LSTM.Word[k][:maxlen]

Ниже те же самые операции только для val

In [129]:
df_val_LSTM = df_val.copy()
df_val_LSTM.Word = df_val.Word.apply(list)
df_val_LSTM.Word = df_val_LSTM.Word.apply(le.transform)

In [130]:
maxlen = 20
X_val_lstm = np.zeros((df_val_LSTM.shape[0], maxlen), dtype=int)
for k in range(df_val_LSTM.shape[0]):
    X_val_lstm[k][:df_val_LSTM.Word[k].shape[0]] = df_val_LSTM.Word[k][:maxlen]

Архитектура LSTM (Была идея использовать BiLSTM, но руки не дошли)

In [188]:
model_LSTM = Sequential()
model_LSTM.add(Embedding(10000, output_dim=64, input_length=X_tr_lstm.shape[1]))
model_LSTM.add(LSTM(32))
model_LSTM.add(Dropout(0.5))
model_LSTM.add(Dense(1, activation='sigmoid'))

model_LSTM.compile(loss='binary_crossentropy',
              optimizer='rmsprop')

Обучение LSTM (принцип обучения такой же как у MLP)

In [189]:
prev_score = 0
for i in range(3):
    is_break = False
    print('global epoch =',i)
    if not is_break:
        for j in range(9):
            print('local epoch =',j)
            if j < 8:
                df_train_downsampled = pd.concat([df_train_label_1, df_train[df_train.Label == 0][j*n:j*n+n]], axis=0)
                df_train_downsampled = df_train_downsampled.sample(frac=1).reset_index(drop=True)
            else:
                df_train_downsampled = pd.concat([df_train_label_1, df_train[df_train.Label == 0][j*n:]], axis=0)
                df_train_downsampled = df_train_downsampled.sample(frac=1).reset_index(drop=True)

            df_train_downsampled.Word = df_train_downsampled.Word.apply(list)
            df_train_downsampled.Word = df_train_downsampled.Word.apply(le.transform)

            maxlen = 20
            X_tr_lstm = np.zeros((df_train_downsampled.shape[0], maxlen), dtype=int)
            for k in range(X_tr_lstm.shape[0]):
                X_tr_lstm[k][:df_train_downsampled.Word[k].shape[0]] = df_train_downsampled.Word[k][:maxlen]

            model_LSTM.fit(X_tr_lstm, df_train_downsampled.Label, batch_size=32, epochs=1)
            score = roc_auc_score(df_val.Label, model_LSTM.predict(X_val_lstm))
            print("val score =", score)
            if score > prev_score:
                prev_score = score
            else:
                print("AUC SCORE DECREASED!!!")
                is_break = True
                break

global epoch = 0
local epoch = 0
Epoch 1/1
18140/18140 [==============================] - 39s 2ms/step - loss: 0.5851
val score = 0.8035865548746469
local epoch = 1
Epoch 1/1
18140/18140 [==============================] - 36s 2ms/step - loss: 0.5133
val score = 0.8426463217731308
local epoch = 2
Epoch 1/1
18140/18140 [==============================] - 35s 2ms/step - loss: 0.4912
val score = 0.8481275804248107
local epoch = 3
Epoch 1/1
18140/18140 [==============================] - 35s 2ms/step - loss: 0.4722
val score = 0.8589451962253795
local epoch = 4
Epoch 1/1
18140/18140 [==============================] - 35s 2ms/step - loss: 0.4664
val score = 0.8637137279730868
local epoch = 5
Epoch 1/1
18140/18140 [==============================] - 36s 2ms/step - loss: 0.4556
val score = 0.871856367787251
local epoch = 6
Epoch 1/1
18140/18140 [==============================] - 36s 2ms/step - loss: 0.4477
val score = 0.8770611590838494
local epoch = 7
Epoch 1/1
18140/18140 [=====================

**Предикт и сабмит**

In [71]:
df_test_part1 = df_test.iloc[0:30000]
df_test_part2 = df_test.iloc[30000:60000]
df_test_part3 = df_test.iloc[60000:90000]
df_test_part4 = df_test.iloc[90000:120000]
df_test_part5 = df_test.iloc[120000:150000]
df_test_part6 = df_test.iloc[150000:180000]
df_test_part7 = df_test.iloc[180000:]

In [72]:
test_part1_vec = vectorizer.transform(df_test_part1.Word)
test_part2_vec = vectorizer.transform(df_test_part2.Word)
test_part3_vec = vectorizer.transform(df_test_part3.Word)
test_part4_vec = vectorizer.transform(df_test_part4.Word)
test_part5_vec = vectorizer.transform(df_test_part5.Word)
test_part6_vec = vectorizer.transform(df_test_part6.Word)
test_part7_vec = vectorizer.transform(df_test_part7.Word)

C:\Users\Alexey\Miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [75]:
MLP_pred = np.zeros((df_test.shape[0],1))

ВАЖНО!!! требуется достаточное кол-во свободной оперативном памяти ~ 10ГБ

In [76]:
MLP_pred[0:30000] = model_MLP.predict(test_part1_vec.toarray())
MLP_pred[30000:60000] = model_MLP.predict(test_part2_vec.toarray())
MLP_pred[60000:90000] = model_MLP.predict(test_part3_vec.toarray())
MLP_pred[90000:120000] = model_MLP.predict(test_part4_vec.toarray())
MLP_pred[120000:150000] = model_MLP.predict(test_part5_vec.toarray())
MLP_pred[150000:180000] = model_MLP.predict(test_part6_vec.toarray())
MLP_pred[180000:] = model_MLP.predict(test_part7_vec.toarray())

Ниже операции с OHE для test

In [190]:
df_test_le = df_test.copy()

In [191]:
df_test_le.Word = df_test.Word.apply(list)
df_test_le.Word = df_test_le.Word.apply(le.transform)

In [192]:
X_test_lstm = np.zeros((df_test.shape[0], maxlen), dtype=int)
for i in range(X_test_lstm.shape[0]):
    X_test_lstm[i][:df_test_le.Word[i].shape[0]] = df_test_le.Word[i][:maxlen]

In [193]:
LSTM_pred = model_LSTM.predict(X_test_lstm)

In [194]:
((MLP_pred + LSTM_pred)/2).mean()

0.24718853454341877

In [195]:
MLP_pred.mean()

0.21580755974812832

In [196]:
LSTM_pred.mean()

0.27856952

Усреднение предсказываения MLP и LSTM

In [161]:
df_subm = pd.read_csv('submission.csv', encoding='utf-8')
df_subm.Prediction = (LSTM_pred + MLP_pred)/2
df_subm.to_csv("submission.csv", encoding='utf-8', index=False)